# 자연어처리 소개

인간의 언어를 컴퓨터와 같은 기계가 묘사할 수 있도록 연구하고 구현하는 분야

: 음성 - Text화 - 말뭉치 사전 기준 토큰화 - 입력벡터 - 딥러닝 모델 처리 - 출력벡터 - 말뭉치 사전 참조 Text 벡터화 - 음성화

# 사전준비 및 라이브러리 설치

In [ ]:
# 한글 형태소 분석기
!pip install konlpy  

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 77.7 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# Sentencepiece 라이브러리 설치
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 13.7 MB/s 


# 1.음성과 Text의 상호변환

## 가.관련 라이브러리 설치 : gTTS, SpeechRecognition

In [ ]:
# text to speech(TTS 음성변환)를 위한 라이브러리
!pip install gTTS

In [ ]:
# speech to text(STT 음성인식) 처리를 위한 라이브러리
!pip3 install SpeechRecognition

     |████████████████████████████████| 32.8 MB 40 kB/s 


## 나.Text를 음성으로 변환: TTS 기술

In [ ]:
from gtts import gTTS
text ="안녕하세요, 여러분! 자연어처리 너무 재미 있지요?"

tts = gTTS(text=text, lang='ko')
tts.save("helloKO.mp3")

다운로드 받아서 들어보세요!

## 다.음성을 Text로 변환 : STT 기술

In [ ]:
!ffmpeg -i helloKO.mp3 mykor.aif

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
import speech_recognition as sr

r = sr.Recognizer()
harvard = sr.AudioFile("mykor.aif")
with harvard as source:
    audio = r.record(source)
print(r.recognize_google(audio, language='ko-KR'))

안녕하세요 여러분 자연어처리 너무 재미있지요


# 2.말뭉치 사전 만들기

말뭉치 사전 : 자연언어 연구를 위해 특정한 목적을 가지고 언어의 표본을 추출한 집합

Text를 수치화 시키는 방법은 말뭉치 사전을 만들고 그것을 기준으로 One-Hot Encoding과 BOW 방법이 있었으나 비효율 적이거나 정보의 유실이 있어 다른 방법을 모색

## 가.말뭉치 사전 만들기

In [ ]:
corpus = """나는 학교에 다니는 학생 입니다
나는 좋은 선생님 입니다
당신은 매우 좋은 선생님 입니다"""

In [ ]:
# 띄어쓰기 기준 토큰으로 분리
tokens = corpus.split()
words = list(dict.fromkeys(tokens))
words

['나는', '학교에', '다니는', '학생', '입니다', '좋은', '선생님', '당신은', '매우']

In [ ]:
# 각 단어에 고유한 번호 부여한 dictionary 생성
word_to_id = {'[PAD]': 0, '[UNK]': 1}  # [PAD]: 길이 맞추는 용도, [UNK]: 알 수 없는 token
for word in words:
    word_to_id[word] = len(word_to_id)
word_to_id

{'[PAD]': 0,
 '[UNK]': 1,
 '나는': 2,
 '다니는': 4,
 '당신은': 9,
 '매우': 10,
 '선생님': 8,
 '입니다': 6,
 '좋은': 7,
 '학교에': 3,
 '학생': 5}

In [ ]:
word_to_id.items()

dict_items([('[PAD]', 0), ('[UNK]', 1), ('나는', 2), ('학교에', 3), ('다니는', 4), ('학생', 5), ('입니다', 6), ('좋은', 7), ('선생님', 8), ('당신은', 9), ('매우', 10)])

In [ ]:
# 고유한 번호로 부터 단어를 찾을 수 있는 dictionary 생성
id_to_word = {id:word for word, id in word_to_id.items()}
id_to_word

{0: '[PAD]',
 1: '[UNK]',
 2: '나는',
 3: '학교에',
 4: '다니는',
 5: '학생',
 6: '입니다',
 7: '좋은',
 8: '선생님',
 9: '당신은',
 10: '매우'}

In [ ]:
# 말뭉치 사전 : word_to_id
corpus_dic = word_to_id
# 말뭉치 인텍스를 기준으로한 사전
corpus_id = id_to_word
# corpus depth 는 길이
len(corpus_dic)

11

* 모든 단어를 포함하는 말뭉치 사전을 만들어서 각 단어에 인텍스를 부여
* 분석하고자 하는 문서를 사전의 고유한 인텍스로 수치화

## 나.One-Hot Encoding

하나의 토큰을 하나의 행벡터로 생성(단어수*사전수)

In [ ]:
text = '''나는 학교에 잘 다니는 학생 입니다
선생님 은 좋은 선생님 입니다'''

In [ ]:
# 줄바꿈 단위로 문장 분리
sentences = text.split("\n")
sentences

['나는 학교에 잘 다니는 학생 입니다', '선생님 은 좋은 선생님 입니다']

In [ ]:
# 띄어쓰기 단위로 단어 분리
tokens = []
for sentence in sentences:
    tokens.append(sentence.split())
tokens

[['나는', '학교에', '잘', '다니는', '학생', '입니다'], ['선생님', '은', '좋은', '선생님', '입니다']]

In [ ]:
# tokens을 말뭉치사전의 고유 번호로 변경
token_ids = []
for line_token in tokens:
  line_ids = []
  for token in line_token:
    if token in words:
      line_ids.append(word_to_id[token])
    else:
      line_ids.append(word_to_id['[UNK]'])
  token_ids.append(line_ids)
print(tokens,'\n',token_ids)

[['나는', '학교에', '잘', '다니는', '학생', '입니다'], ['선생님', '은', '좋은', '선생님', '입니다']] 
 [[2, 3, 1, 4, 5, 6], [8, 1, 7, 8, 6]]


In [ ]:
# one hot encoding
one_hot_encodings = []
for line_token in token_ids:
    print(line_token)
    one_hot_line = []  # 한 줄을 표현하는 벡터
    for id in line_token:
        # print(id)
        one_hot = [0] * len(word_to_id)  # 모두 0인 벡터를 만듬
        # print(one_hot)
        one_hot[id] = 1  # 단어 id만 1로 변경
        print(id, one_hot)
        one_hot_line.append(one_hot)  # 단어를 라인에 추가
    print(one_hot_line)
    one_hot_encodings.append(one_hot_line)  # 라인을 전체 문서에 추가

[2, 3, 1, 4, 5, 6]
2 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
1 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
5 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
6 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]
[8, 1, 7, 8, 6]
8 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
1 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
8 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
6 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]


In [ ]:
# one-hot의 행렬의 구조는 문장수 * corpus_dic의 길이(열) * 토큰수(행)
one_hot_encodings

[[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]]

In [ ]:
import numpy as np
# argmax는 max를 가지고 있는 index를 리턴, axis 개념 이해 필요
# shape 행렬수*열수*행수
a = np.arange(24).reshape(2,3,4)
print(a)
print(np.argmax(a, axis=0))
print(np.argmax(a, axis=1))
print(np.argmax(a, axis=2))
print(np.argmax(a, axis=-1))


[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
[[2 2 2 2]
 [2 2 2 2]]
[[3 3 3]
 [3 3 3]]
[[3 3 3]
 [3 3 3]]


In [ ]:
# help(np.argmax)

In [ ]:
# 행기준으로 one-hot을 찾아냄
import numpy as np
print(tokens[0])
np.argmax(np.array(one_hot_encodings[0]), axis=-1)

['나는', '학교에', '잘', '다니는', '학생', '입니다']


array([2, 3, 1, 4, 5, 6])

In [ ]:
# tf를 이용한 one-hot encoding, 조건: 문장들의 길이(rows)가 동일해야함 
import tensorflow as tf
# 열의 길이는 depth, corpus_dic 길이

tf_one_hot_encodings = tf.one_hot(indices=token_ids[0], depth=len(word_to_id))
tf_one_hot_encodings

<tf.Tensor: shape=(6, 11), dtype=float32, numpy=
array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)>

In [ ]:
# 문장들의 길이가 rows = 6 되도록 pad를 추가
pad_ids = []
rows=6
for line in token_ids:
    line = line[:rows]
    line += [0] * (rows - len(line))
    pad_ids.append(line)
pad_ids

[[2, 3, 1, 4, 5, 6], [8, 1, 7, 8, 6, 0]]

In [ ]:
#기존 one-hot에서 마지막 행 pad가 추가된 one-hot
tf_one_hot_encodings = tf.one_hot(indices=pad_ids, depth=len(word_to_id))
tf_one_hot_encodings

<tf.Tensor: shape=(2, 6, 11), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>

In [ ]:
# [나는 학교에 잘 다니는 학생 입니다]의 one-hot encoding
import pandas as pd

df = pd.DataFrame(np.int8(tf_one_hot_encodings[1]), columns=corpus_dic.keys())
df

,[PAD],[UNK],나는,학교에,다니는,학생,입니다,좋은,선생님,당신은,매우
0,0,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,0


## 다.BOW 생성

말뭉치 사전의 토큰 인덱스 위치에 토큰 출현 회수 누적, 순서 정보의 손실

In [ ]:
sentences

['나는 학교에 잘 다니는 학생 입니다', '선생님 은 좋은 선생님 입니다']

In [ ]:
# sentences의 one-hot으로부터 열 기준합계로 Bow 생성
sentence = np.sum(tf_one_hot_encodings, axis=1)
sentence

array([[0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 1., 1., 2., 0., 0.]], dtype=float32)

In [ ]:
# [나는 학교에 잘 다니는 학생 입니다, 선생님 은 좋은 선생님 입니다]의 BOW 표현
import pandas as pd
df = pd.DataFrame(np.int8(sentence), columns=corpus_dic.keys())
df

,[PAD],[UNK],나는,학교에,다니는,학생,입니다,좋은,선생님,당신은,매우
0,0,1,1,1,1,1,1,0,0,0,0
1,1,1,0,0,0,0,1,1,2,0,0


# 3.다양한 형태의 토큰화

토큰화 : 일정한 입력단위로 분할

## 가.문자나 단어 단위 토큰화

In [ ]:
corpus

'나는 학교에 다니는 학생 입니다\n나는 좋은 선생님 입니다\n당신은 매우 좋은 선생님 입니다'

In [ ]:
# 문자단위로 나누기: list
print(list(corpus))

['나', '는', ' ', '학', '교', '에', ' ', '다', '니', '는', ' ', '학', '생', ' ', '입', '니', '다', '\n', '나', '는', ' ', '좋', '은', ' ', '선', '생', '님', ' ', '입', '니', '다', '\n', '당', '신', '은', ' ', '매', '우', ' ', '좋', '은', ' ', '선', '생', '님', ' ', '입', '니', '다']


In [ ]:
# space 기준 단어단위 나누기 : split
print(corpus.split())

['나는', '학교에', '다니는', '학생', '입니다', '나는', '좋은', '선생님', '입니다', '당신은', '매우', '좋은', '선생님', '입니다']


## 나.형태소 기준 토큰화

In [ ]:
import konlpy
komoran = konlpy.tag.Komoran()
komoran.morphs('아버지가방에들어가신다')

['아버지', '가방', '에', '들어가', '시', 'ㄴ다']

In [ ]:
print(komoran.morphs(corpus))

['나', '는', '학교', '에', '다니', '는', '학생', '이', 'ㅂ니다', '나', '는', '좋', '은', '선생님', '이', 'ㅂ니다', '당신', '은', '매우', '좋', '은', '선생님', '이', 'ㅂ니다']


In [ ]:
text = '홍길동 전화번호는 Kt 1234-5678번 입니까?'

In [ ]:
# 한글 형태소분석기 클래스 불러오기
from konlpy.tag import Hannanum, Komoran, Kkma, Okt

komoran = Komoran()
print(komoran.pos(text))

okt = Okt()
print(okt.pos(text))

[('홍길동', 'NNP'), ('전화번호', 'NNP'), ('는', 'JX'), ('Kt', 'SL'), ('1234', 'SN'), ('-', 'SW'), ('5678', 'SN'), ('번', 'NNB'), ('입', 'VV'), ('니까', 'EF'), ('?', 'SF')]
[('홍길동', 'Noun'), ('전화번호', 'Noun'), ('는', 'Josa'), ('Kt', 'Alpha'), ('1234-5678', 'Number'), ('번', 'Noun'), ('입', 'Noun'), ('니까', 'Josa'), ('?', 'Punctuation')]


In [ ]:
# komoran.tagset

In [ ]:
# tagset 보기
print(komoran.tagset)

{'EC': '연결 어미', 'EF': '종결 어미', 'EP': '선어말어미', 'ETM': '관형형 전성 어미', 'ETN': '명사형 전성 어미', 'IC': '감탄사', 'JC': '접속 조사', 'JKB': '부사격 조사', 'JKC': '보격 조사', 'JKG': '관형격 조사', 'JKO': '목적격 조사', 'JKQ': '인용격 조사', 'JKS': '주격 조사', 'JKV': '호격 조사', 'JX': '보조사', 'MAG': '일반 부사', 'MAJ': '접속 부사', 'MM': '관형사', 'NA': '분석불능범주', 'NF': '명사추정범주', 'NNB': '의존 명사', 'NNG': '일반 명사', 'NNP': '고유 명사', 'NP': '대명사', 'NR': '수사', 'NV': '용언추정범주', 'SE': '줄임표', 'SF': '마침표, 물음표, 느낌표', 'SH': '한자', 'SL': '외국어', 'SN': '숫자', 'SO': '붙임표(물결,숨김,빠짐)', 'SP': '쉼표,가운뎃점,콜론,빗금', 'SS': '따옴표,괄호표,줄표', 'SW': '기타기호 (논리수학기호,화폐기호)', 'VA': '형용사', 'VCN': '부정 지정사', 'VCP': '긍정 지정사', 'VV': '동사', 'VX': '보조 용언', 'XPN': '체언 접두사', 'XR': '어근', 'XSA': '형용사 파생 접미사', 'XSN': '명사파생 접미사', 'XSV': '동사 파생 접미사'}


In [ ]:
# 형태소 분석
print(komoran.morphs(text))
print(okt.morphs(text))

['홍길동', '전화번호', '는', 'Kt', '1234', '-', '5678', '번', '입', '니까', '?']
['홍길동', '전화번호', '는', 'Kt', '1234-5678', '번', '입', '니까', '?']


In [ ]:
# 명사추출
print(komoran.nouns(text))
print(okt.nouns(text))

['홍길동', '전화번호', '번']
['홍길동', '전화번호', '번', '입']


## 다.토큰 다루는 여러가지 방법

In [ ]:
# 토큰 count
import collections
char_counter = collections.Counter()
char_counter.update(list(corpus))

In [ ]:
corpus

'나는 학교에 다니는 학생 입니다\n나는 좋은 선생님 입니다\n당신은 매우 좋은 선생님 입니다'

In [ ]:
print(char_counter)

Counter({' ': 11, '다': 4, '니': 4, '는': 3, '생': 3, '입': 3, '은': 3, '나': 2, '학': 2, '\n': 2, '좋': 2, '선': 2, '님': 2, '교': 1, '에': 1, '당': 1, '신': 1, '매': 1, '우': 1})


In [ ]:
# 유니크한 char 개수
print(len(char_counter))

19


In [ ]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



In [ ]:
# 빈도수가 많은 글자 10개 출력
most_freq = sorted(char_counter.items(), key=lambda item: item[1], reverse=True)
print(most_freq[:10])

[(' ', 11), ('다', 4), ('니', 4), ('는', 3), ('생', 3), ('입', 3), ('은', 3), ('나', 2), ('학', 2), ('\n', 2)]


In [ ]:
def get_key(item):
    return item[1]

most_freq = sorted(char_counter.items(), key=get_key, reverse=True)
print(most_freq[:10])

[(' ', 11), ('다', 4), ('니', 4), ('는', 3), ('생', 3), ('입', 3), ('은', 3), ('나', 2), ('학', 2), ('\n', 2)]


In [ ]:
# 빈도수가 적은 글자 10개 출력
least_freq = sorted(char_counter.items(), key=lambda item: item[1])
print(least_freq[:10])

[('교', 1), ('에', 1), ('당', 1), ('신', 1), ('매', 1), ('우', 1), ('나', 2), ('학', 2), ('\n', 2), ('좋', 2)]


In [ ]:
# 한글 문자를 참조 인덱스로 변환 함수 ord, 참조인덱스를 한글로 변환 chr 
print(ord('가'),ord('힣'), chr(44032), chr(44034))
# help(ord)

44032 55203 가 갂


In [ ]:
hangul = [chr(i) for i in range(ord('가'), ord('힣') + 1)]
len(hangul)

11172

In [ ]:
print(hangul[:100])

['가', '각', '갂', '갃', '간', '갅', '갆', '갇', '갈', '갉', '갊', '갋', '갌', '갍', '갎', '갏', '감', '갑', '값', '갓', '갔', '강', '갖', '갗', '갘', '같', '갚', '갛', '개', '객', '갞', '갟', '갠', '갡', '갢', '갣', '갤', '갥', '갦', '갧', '갨', '갩', '갪', '갫', '갬', '갭', '갮', '갯', '갰', '갱', '갲', '갳', '갴', '갵', '갶', '갷', '갸', '갹', '갺', '갻', '갼', '갽', '갾', '갿', '걀', '걁', '걂', '걃', '걄', '걅', '걆', '걇', '걈', '걉', '걊', '걋', '걌', '걍', '걎', '걏', '걐', '걑', '걒', '걓', '걔', '걕', '걖', '걗', '걘', '걙', '걚', '걛', '걜', '걝', '걞', '걟', '걠', '걡', '걢', '걣']


In [ ]:
print(hangul[-100:])

['흀', '흁', '흂', '흃', '흄', '흅', '흆', '흇', '흈', '흉', '흊', '흋', '흌', '흍', '흎', '흏', '흐', '흑', '흒', '흓', '흔', '흕', '흖', '흗', '흘', '흙', '흚', '흛', '흜', '흝', '흞', '흟', '흠', '흡', '흢', '흣', '흤', '흥', '흦', '흧', '흨', '흩', '흪', '흫', '희', '흭', '흮', '흯', '흰', '흱', '흲', '흳', '흴', '흵', '흶', '흷', '흸', '흹', '흺', '흻', '흼', '흽', '흾', '흿', '힀', '힁', '힂', '힃', '힄', '힅', '힆', '힇', '히', '힉', '힊', '힋', '힌', '힍', '힎', '힏', '힐', '힑', '힒', '힓', '힔', '힕', '힖', '힗', '힘', '힙', '힚', '힛', '힜', '힝', '힞', '힟', '힠', '힡', '힢', '힣']


# 4.빅데이터 한국어 말뭉치 사전 개발

말뭉치 사전을 만드는 가장 좋은 방법은 한국어 형태소분석기와 Sentencepiece를 동시에 사용하는 것으로 알려짐 

한국어 wiki 말뭉치 : https://dumps.wikimedia.org/kowiki/ 

## 가.한국어 위키 데이터 말뭉치

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data dir
import os
data_dir = '/content/drive/MyDrive/Data'
os.listdir(data_dir)

['kowiki']

In [ ]:
import os
corpus = os.path.join(data_dir, "kowiki", "kowiki.txt.zip")
corpus_dir = os.path.dirname(corpus)
corpus

'/content/drive/MyDrive/Data/kowiki/kowiki.txt.zip'

In [ ]:
# wiki 라인수 확인, tqdm 함수 사용시 전체 진행 경과 확인에 필요
import zipfile
from tqdm.notebook import tqdm, trange

count = 0
with zipfile.ZipFile(corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(tqdm(f)):
            count += 1
print(count)

0it [00:00, ?it/s]

3724301


In [ ]:
# wiki 내용 확인
with zipfile.ZipFile(corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 50:
                break
            line = line.decode('utf-8').strip()
            print(line)

지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.
그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 결국 재선에 실패했다. 또한 임기 말기에 터진 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 내세웠다.
지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통

## 나.Sentencepiece 라이브러리

sentencepiece 라이브러리는 우리가 지정해주는 사전의 크기를 기준으로 단어를 문자 단위로 분해하고, 높은 빈도수 순으로 subword를 생성해서 토큰을 만들어 단어사전을 만든다.

In [ ]:
import sentencepiece as spm
def train_sentencepiece(corpus, prefix, vocab_size=32000):
    """
    sentencepiece를 이용해 vocab 학습
    :param corpus: 학습할 말뭉치
    :param prefix: 저장할 vocab 이름
    :param vocab_size: vocab 개수
    """
    spm.SentencePieceTrainer.train(
        f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +  # 7은 특수문자 개수
        " --model_type=unigram" +
        " --max_sentence_length=999999" +  # 문장 최대 길이
        " --pad_id=0 --pad_piece=[PAD]" +  # pad token 및 id 지정
        " --unk_id=1 --unk_piece=[UNK]" +  # unknown token 및 id 지정
        " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence token 및 id 지정
        " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence token 및 id 지정
        " --user_defined_symbols=[SEP],[CLS],[MASK]" +  # 기타 추가 토큰 SEP: 4, CLS: 5, MASK: 6
        " --input_sentence_size=100000" +  # 말뭉치에서 셈플링해서 학습
        " --shuffle_input_sentence=true")  # 셈플링한 말뭉치 shuffle

In [ ]:
# 현재 작업폴더에 압축풀기
!unzip /content/drive/MyDrive/Data/kowiki/kowiki.txt.zip

print(os.listdir("./"))

Archive:  /content/drive/MyDrive/Data/kowiki/kowiki.txt.zip
  inflating: kowiki.txt              
  inflating: __MACOSX/._kowiki.txt   
['.config', 'drive', 'mykor.aif', 'kowiki.txt', '__MACOSX', 'helloKO.mp3', 'sample_data']


In [ ]:
# 말뭉치 사전 생성
train_sentencepiece("kowiki.txt", "kowiki_32000")

In [ ]:
# 생성결과 확인
print(os.listdir("./"))

['.config', 'drive', 'mykor.aif', 'kowiki.txt', '__MACOSX', 'helloKO.mp3', 'kowiki_32000.model', 'kowiki_32000.vocab', 'sample_data']


In [ ]:
# 생성파일 백업, shutil은 파일 복제 복사 라이브러리
import shutil
corpus_dir = os.path.dirname(corpus)
shutil.copy("kowiki_32000.model", corpus_dir)
shutil.copy("kowiki_32000.vocab", corpus_dir)
shutil.copy("kowiki.txt", corpus_dir)

print(os.listdir(corpus_dir))

['kowiki.txt.zip', 'kowiki-mecab.txt', 'kowiki.txt', 'kowiki_32000.model', 'kowiki_32000.vocab', 'kowiki_mecab_32000.model', 'kowiki_mecab_32000.vocab']


In [ ]:
# 말뭉치 사전 업로드
spm_vocab = spm.SentencePieceProcessor()
spm_vocab.load(os.path.join(corpus_dir,"kowiki_32000.model"))

True

In [ ]:
# 말뭉치 사전 기준 토큰화
with zipfile.ZipFile(corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            tokens = spm_vocab.encode_as_pieces(line)
            print(tokens)
            _ids = spm_vocab.encode_as_ids(line)
            print(_ids)

지미 카터
['▁지미', '▁카터']
[14470, 19081]
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
['▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카터', '▁주니어', '(', ',', '▁1924', '년', '▁10', '월', '▁1', '일', '▁~', '▁)', '는', '▁민주당', '▁출신', '▁미국', '▁39', '번째', '▁대통령', '▁', '(1977', '년', '▁~', '▁1981', '년', ')', '이다', '.']
[2002, 5517, 49, 45, 167, 50, 19081, 5795, 16, 9, 5556, 17, 67, 24, 36, 30, 86, 140, 12, 2473, 702, 139, 5020, 672, 654, 10, 26492, 17, 86, 2275, 17, 13, 26, 7]
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
['▁지미', '▁카터', '는', '▁조지아주', '▁섬', '터', '▁카운티', '▁플레', '인', '스', '▁마을에서', '▁태어났다', '.', '▁조지아', '▁공과대학교', '를', '▁졸업', '하였다', '.', '▁그', '▁후', '▁해군', '에', '▁들어가', '▁전함', '·', '원자력', '·', '잠', '수', '함', '의', '▁승무원', '으로', '▁일', '하였다', '.', '▁1953', '년', '▁미국', '▁해군', '▁대위', '로', '▁예편', '하였고', '▁이후', '▁땅콩', '·', '면', '화', '▁등을', '▁가', '꿔', 

In [ ]:
# 문자열을 token으로 분할: encode_as_pieces
tokens = spm_vocab.encode_as_pieces("아름다운 대한민국 우리나라 금수강산")

print(tokens)

['▁아름다운', '▁대한민국', '▁우리나라', '▁금', '수', '강', '산']


In [ ]:
# token을 문자열로 복원: decode_pieces 
print(spm_vocab.decode_pieces(tokens))

아름다운 대한민국 우리나라 금수강산


In [ ]:
# 문자열을 숫자로 분할 : encode_as_ids
ids = spm_vocab.encode_as_ids("아름다운 대한민국 우리나라 금수강산")

print(ids)

[4877, 243, 6310, 653, 103, 299, 158]


In [ ]:
# 숫자를 문자열로 복원
print(spm_vocab.decode_ids(ids))

아름다운 대한민국 우리나라 금수강산


In [ ]:
# token을 숫자로 변경
print(spm_vocab.piece_to_id(tokens))

[4877, 243, 6310, 653, 103, 299, 158]


In [ ]:
# 숫자를 token으로 변경
print(spm_vocab.id_to_piece(ids))

['▁아름다운', '▁대한민국', '▁우리나라', '▁금', '수', '강', '산']


## 다.형태소분석을 이용한 말뭉치 사전 개발

In [ ]:
# 한글 형태소 분석기 mecab 설치 : 4분정도 소요
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4 MB 22.6 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 448 kB 54.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/

In [ ]:
# 대용량 처리는 Mecab 사용 권장
from konlpy.tag import Mecab
mecab = Mecab() 
' '.join(mecab.morphs("아버지가방에들어가신다"))

'아버지 가 방 에 들어가 신다'

In [ ]:
# morph 단위로 분할된 말뭉치 생성,  mecab으로 진행시 20분 정도 소요 
from tqdm.notebook import tqdm, trange
import zipfile

with open("kowiki-mecab.txt", "w") as o_f:
    with zipfile.ZipFile(corpus) as z:
        with z.open('kowiki.txt') as f:
            for i, line in enumerate(tqdm(f, total=count)):
                line = line.decode('utf-8').strip()
                tokens = mecab.morphs(line)
                string = " ".join(tokens)
                o_f.write(string)
                o_f.write("\n")

  0%|          | 0/3724301 [00:00<?, ?it/s]

In [ ]:
# 파일 확인
print(os.listdir("./"))

['.config', 'kowiki-okt.txt', 'kowiki-mecab.txt', 'drive', 'sample_data']


In [ ]:
import shutil
corpus_dir = os.path.dirname(corpus)
shutil.copy("kowiki-mecab.txt", corpus_dir)

'/content/drive/MyDrive/Data/kowiki/kowiki-mecab.txt'

In [ ]:
# 기존 kowiki
with open(os.path.join(corpus_dir,'kowiki.txt')) as f:
    for i, line in enumerate(f):
        if i > 5:
            break
        print(line.strip())
# Mecab 형태소 분석한 kowiki
with open(os.path.join(corpus_dir,'kowiki-mecab.txt')) as f:
    for i, line in enumerate(f):
        if i > 5:
            break
        print(line.strip())

지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
지미 카터
제임스 얼 " 지미 " 카터 주니어 ( , 1924 년 10 월 1 일 ~ ) 는 민주당 출신 미국 39 번 째 대통령 ( 1977 년 ~ 1981 년 ) 이 다 .
지미 카터 는 조지 아주 섬터 카운티 플 레인스 마을 에서 태어났 다 . 조지 아 공과 대학교 를 졸업 하 였 다 . 그 후 해군 에 들어가 전함 · 원자력 · 잠수함 의 승무원 으로 일 하 였 다 . 1953 년 미국 해군 대위 로 예편 하 였 고 이후 땅콩 · 면화 등 을 가꿔 많 은 돈 을 벌 었 다 . 그 의 별명 이 " 땅콩 농부 " ( Peanut Farmer ) 로 알려졌 다 .
1962 년 조지아 주 상원 의원 선거 에서 낙선 하 나 그 선거 가 부정 선거 였음을 입증 하 게 되 어 당선 되 고 , 1966 년 조지아 주 지사 선거 에 낙선 하 지만 1970 년 조지아 주 지사 를 역임 했 다 . 대통

In [ ]:
# morph vocab 학습
train_sentencepiece(os.path.join(corpus_dir,'kowiki-mecab.txt'), "kowiki_mecab_32000")

In [ ]:
# 생성결과 확인
print(os.listdir("./"))

['.config', 'kowiki_mecab_32000.vocab', 'drive', 'mykor.aif', 'kowiki_mecab_32000.model', 'kowiki.txt', '__MACOSX', 'helloKO.mp3', 'kowiki_32000.model', 'kowiki_32000.vocab', 'sample_data']


In [ ]:
# 생성파일 백업
corpus_dir = os.path.dirname(corpus)
shutil.copy("kowiki_mecab_32000.model", corpus_dir)
shutil.copy("kowiki_mecab_32000.vocab", corpus_dir)

print(os.listdir(corpus_dir))

['kowiki.txt.zip', 'kowiki-mecab.txt', 'kowiki.txt', 'kowiki_32000.model', 'kowiki_32000.vocab', 'kowiki_mecab_32000.model', 'kowiki_mecab_32000.vocab']


In [ ]:
# load morph vocab
spm_morph_vocab = spm.SentencePieceProcessor()
spm_morph_vocab.load(os.path.join(corpus_dir, "kowiki_mecab_32000.model"))

True

In [ ]:
# wiki spm tokenize
with zipfile.ZipFile(corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            tokens = spm_morph_vocab.encode_as_pieces(line)
            print(tokens)
            _ids = spm_morph_vocab.encode_as_ids(line)
            print(_ids)

지미 카터
['▁지', '미', '▁카터']
[45, 520, 8873]
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
['▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카터', '▁주니어', '(', ',', '▁19', '24', '년', '▁10', '월', '▁1', '일', '▁~', '▁)', '는', '▁', '민주당', '▁출신', '▁미국', '▁39', '번', '째', '▁대통령', '▁(', '19', '77', '년', '▁~', '▁19', '81', '년', ')', '이', '다', '.']
[2351, 3479, 50, 75, 520, 4585, 8873, 4349, 2303, 2379, 46, 2265, 561, 110, 1917, 44, 519, 102, 18, 746, 16, 2003, 594, 125, 2837, 1914, 2135, 457, 19, 1880, 4281, 561, 102, 46, 2521, 561, 3614, 71, 220, 2543]
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
['▁지', '미', '▁카터', '는', '▁조지아', '주', '▁섬', '터', '▁카운티', '▁플레인', '스', '▁마을', '에', '서', '▁태어났', '다', '.', '▁조지아', '▁공과', '대', '학교', '를', '▁졸업', '하', '였', '다', '.', '▁그', '▁후', '▁해군', '에', '▁들', '어가', '▁전함', '·', '원', '자', '력', '·', '잠', '수', '함', '의', '▁승무', '원',

## 라.개발 사전 비교

In [ ]:
# 말뭉치 사전 업로드
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load(os.path.join(data_dir,'kowiki', "kowiki_32000.model"))
sp_morph = spm.SentencePieceProcessor()
sp_morph.load(os.path.join(data_dir,'kowiki', "kowiki_mecab_32000.model"))

True

In [ ]:
# wiki spm tokenize
import zipfile
with zipfile.ZipFile(corpus) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            tokens = sp.encode_as_pieces(line)
            print(tokens)
            # _ids = sp.encode_as_ids(line)
            # print(_ids)
            tokens1 = sp_morph.encode_as_pieces(line)
            print(tokens1)
            # _ids1 = sp_morph.encode_as_ids(line)
            # print(_ids1)
            print()
            

지미 카터
['▁지미', '▁카터']
['▁지', '미', '▁카터']

제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
['▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카터', '▁주니어', '(', ',', '▁1924', '년', '▁10', '월', '▁1', '일', '▁~', '▁)', '는', '▁민주당', '▁출신', '▁미국', '▁39', '번째', '▁대통령', '▁', '(1977', '년', '▁~', '▁1981', '년', ')', '이다', '.']
['▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카터', '▁주니어', '(', ',', '▁19', '24', '년', '▁10', '월', '▁1', '일', '▁~', '▁)', '는', '▁', '민주당', '▁출신', '▁미국', '▁39', '번', '째', '▁대통령', '▁(', '19', '77', '년', '▁~', '▁19', '81', '년', ')', '이', '다', '.']

지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
['▁지미', '▁카터', '는', '▁조지아주', '▁섬', '터', '▁카운티', '▁플레', '인', '스', '▁마을에서', '▁태어났다', '.', '▁조지아', '▁공과대학교', '를', '▁졸업', '하였다', '.', '▁그', '▁후', '▁해군', '에', '▁들어가', '▁전함', '·', '원자력', '·', '잠', '수', '함', '의', '▁승무원', '으로', '▁일', '하였다', '.', '▁1953', '년', '▁미국'

In [ ]:
corpus = """나는 학교에 다니는 학생 입니다
나는 좋은 선생님 입니다
당신은 매우 좋은 선생님 입니다"""

In [ ]:
print(corpus)
print(sp.encode_as_pieces(corpus))
print(sp_morph.encode_as_pieces(corpus))
print(sp_morph.encode_as_ids(corpus))

나는 학교에 다니는 학생 입니다
나는 좋은 선생님 입니다
당신은 매우 좋은 선생님 입니다
['▁나는', '▁학교에', '▁다니는', '▁학생', '▁', '입니다', '▁나는', '▁좋은', '▁선생님', '▁', '입니다', '▁당신', '은', '▁매우', '▁좋은', '▁선생님', '▁', '입니다']
['▁나', '는', '▁학교', '에', '▁다니', '는', '▁학생', '▁입니다', '▁나', '는', '▁좋', '은', '▁선생', '님', '▁입니다', '▁당', '신', '은', '▁매우', '▁좋', '은', '▁선생', '님', '▁입니다']
[79, 746, 200, 711, 2821, 746, 911, 3675, 79, 746, 602, 509, 2325, 1320, 3675, 244, 402, 509, 585, 602, 509, 2325, 1320, 3675]


In [ ]:
# vocab 사이즈를 출력합니다.
print(sp_morph.get_piece_size())

32007


In [ ]:
# 모두 출력 
for id in range(20):
  print(sp_morph.id_to_piece(id), sp.is_control(id))

[PAD] True
[UNK] False
[BOS] True
[EOS] True
[SEP] False
[CLS] False
[MASK] False
▁. False
▁의 False
▁다 False
▁이 False
▁는 False
▁, False
▁에 False
▁을 False
▁하 False
▁ False
▁은 False
▁) False
▁( False


In [ ]:
# 아이디는 piece로 piece는 아이디로 반환합니다.
print(sp_morph.id_to_piece(9))
print(sp_morph.piece_to_id('▁나'))

# 알수 없는 토큰의 겨우 1을 반환합니다.
print(sp_morph.piece_to_id('__MUST_BE_UNKNOWN__'))

▁다
79
1


In [ ]:
# morph 개수
tokens1 = sp.encode_as_pieces(corpus)
tokens2 = sp_morph.encode_as_pieces(corpus)
print(len(tokens1),len(tokens2))

18 24
